In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
search= TavilySearchResults()

In [4]:
search.invoke("What is the status of olympics 2024?")

[{'url': 'https://www.cbssports.com/olympics/news/2024-paris-olympics-updates-results-tracker-team-usa-schedule-united-states-medal-count-from-summer-games/live/',
  'content': "Sunday action at the 2024 Paris Olympics is already underway, with one of the top headlines being Simone Biles' return to Olympic competition, LeBron James leading Team USA against Nikola Jokic's ..."},
 {'url': 'https://olympics.com/ioc/paris-2024',
  'content': 'Paris 2024 will herald a "new era" of Games that will set a precedent for future editions. Find out here how the IOC supports the Organising Committee and get access to key documents like the Host City Contract, qualification information, anti-doping and medical rules as well as media guidelines and News Access Rules.'},
 {'url': 'https://www.bbc.co.uk/sport/articles/cz99g6g8m92o',
  'content': 'The Paris 2024 Olympic Games will showcase 3,800 hours of live sport and award 329 gold medals in 32 sports over 18 days of competition across the capital and

### We have one tool created.
* We will create one more tool which is a retriever which retrieves data from an external source

### Retriever
* We will create retriever with our own data

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [28]:
## Load data from a website 
loader= WebBaseLoader("https://docs.smith.langchain.com/overview")
docs= loader.load()

In [29]:
### Split the text into chunks
documents= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

In [30]:
### Push docs to Vector DB
vector= FAISS.from_documents(documents, OpenAIEmbeddings())
retriever= vector.as_retriever()

In [31]:
from pprint import pprint

In [32]:
pprint(retriever)

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127a44d30>)


In [34]:
retriever.invoke("How to upload dataset?")

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name

#### Now this is a retriever.We will create retriever tool

In [36]:
from langchain.tools.retriever import create_retriever_tool

In [37]:
retriever_tool= create_retriever_tool(
    retriever,
    "langsmith_search",
    "Provides information about langsmith and its components"
)

### Now we have got Tools required. 
* Search tool
* Vector database

Now we will create agent with these two tools. But before that we will create list for tools

In [38]:
tools= [search, retriever_tool]

### We have got the tools, now we will create agent
* To create agent we will append LLM which takes input and decides which tool is suitable to answer
Below are the requirements
* LLM
* Tools
* Prompt

In [39]:
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

#### Load sample prompt

In [40]:
from langchain import hub
##Sample prompt
prompt= hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Create Agent now.

In [41]:
from langchain.agents import create_tool_calling_agent
agent= create_tool_calling_agent(llm, tools, prompt)

### Agent Created!
### Now we have to execute the agent, for that we will call agent executer

In [42]:
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent, tools=tools, verbose=True)

#### Now we have Agent executor. We will invoke a message and check how will it react

In [46]:
agent_executor.invoke({"input": "Hello!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hello!', 'output': 'Hello! How can I assist you today?'}

In [47]:
agent_executor.invoke({"input": "How to make egg omlette?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'How to make egg omelette'}`


[{'url': 'https://www.simplyrecipes.com/recipes/how_to_make_an_omelet/', 'content': 'Simply Recipes / Sally Vargas\nMore Classic Ways to Make Eggs\nHow to Make an Omelette\nIngredients\n2 large eggs\nPinch salt\n1 tablespoon unsalted butter\n2 tablespoons grated cheese, any kind\n3 to 4 cherry tomatoes, cut in half and sprinkled lightly with salt\n2 tablespoons chopped basil, parsley, or herb of your choice\nMethod\nCrack the eggs into a bowl. Simply Recipes / Sally Vargas\nSimply Recipes / Sally Vargas\nPlace the cheese, tomatoes, and herbs in a line down the center of the omelette and cook for about 1 minute longer, or until the eggs are mostly set but still a little soft in the center.\n An American omelette (or "omelet" as they are sometimes spelled) starts out in the same way, but as the eggs cook, the edges are lifted from the sides of the pan with a spat

{'input': 'How to make egg omlette?',
 'output': "I found several resources that can help you make an egg omelette:\n\n1. [Simply Recipes - How to Make an Omelette](https://www.simplyrecipes.com/recipes/how_to_make_an_omelet/): This recipe includes ingredients like eggs, salt, butter, cheese, tomatoes, and herbs. It provides a step-by-step method to make a classic omelette.\n\n2. [Natasha's Kitchen - Perfect Omelette Recipe](https://natashaskitchen.com/perfect-omelette-recipe/): This recipe guides you on how to make moist and fluffy omelettes every time. It includes beating eggs with salt, heating a skillet, and cooking the omelette.\n\n3. [The Kitchn - Omelet Recipe](https://www.thekitchn.com/omelet-recipe-23395435): This recipe suggests cracking eggs into a bowl, adding salt and pepper, heating a nonstick pan, and cooking the omelette with butter.\n\n4. [Food Network - How to Make the Perfect Omelet](https://www.foodnetwork.com/how-to/articles/how-to-make-the-perfect-omelet-a-step-by

In [48]:
agent_executor.invoke({"input": "How to create langsmith dataset?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'How to create langsmith dataset?'}`


Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We provide multipl

{'input': 'How to create langsmith dataset?',
 'output': 'To create a LangSmith dataset, you can follow these steps:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langsmith`\n\n2. Create an API key:\n   - Go to the Settings page and click on "Create API Key."\n\n3. Set up your environment:\n   - Shell commands:\n     ```\n     export LANGCHAIN_TRACING_V2=true\n     export LANGCHAIN_API_KEY=<your-api-key>\n     export OPENAI_API_KEY=<your-openai-api-key>\n     ```\n\n4. Log your first trace:\n   - You can define your dataset and examples using the provided code snippets in Python or TypeScript.\n\n5. Define your evaluator:\n   - Implement an evaluator function to evaluate the outputs of your model against the expected outputs.\n\nBy following these steps, you can create a LangSmith dataset and start evaluating your language model applications.'}

### Now LLM will automatically assign the tasks to respective tool.

### Lets Add memory to the agent to give better understanding of the context

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [50]:
message_history= ChatMessageHistory()

In [52]:
agent_with_chat_history= RunnableWithMessageHistory(
    agent_executor,
    ### Session id is required for users
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [53]:
agent_with_chat_history.invoke({"input":"Hello I'm Vicky"},
                               config={"configurable": {"session_id":"Vicky"}})



> Entering new AgentExecutor chain...
Hello Vicky! How can I assist you today?

> Finished chain.


{'input': "Hello I'm Vicky",
 'chat_history': [],
 'output': 'Hello Vicky! How can I assist you today?'}

In [54]:
agent_with_chat_history.invoke({"input":"What is my name?"},
                               config={"configurable": {"session_id":"Vicky"}})



> Entering new AgentExecutor chain...
Your name is Vicky. How can I help you, Vicky?

> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content="Hello I'm Vicky"),
  AIMessage(content='Hello Vicky! How can I assist you today?')],
 'output': 'Your name is Vicky. How can I help you, Vicky?'}

In [56]:
agent_with_chat_history.invoke({"input":"How many medals did India won in olympics 2024?"},
                               config={"configurable": {"session_id":"Vicky"}})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'India medals won in Olympics 2024'}`


[{'url': 'https://www.sportingnews.com/in/other-sports/news/india-medals-olympics-full-list-medal-winners-standings-paris-2024/5a0d25a5d8ab4597fdb0e9d9', 'content': "India medal winners at Paris Olympics 2024. Manu Bhaker wins India's first medal at the Paris Olympics 2024. The 22-year-old shooter struck Bronze for India, to become the first woman to win a ..."}, {'url': 'https://olympics.com/en/news/paris-2024-olympics-india-medals-tally-winners-table', 'content': "Manu Bhaker won the first medal for India at the Paris 2024 Olympics. She bagged a brone and became the first Indian woman to win an Olympic shooting medal. Overall, India have won 36 medals at the Olympics to date. Interestingly, it was Norman Pritchard's dual silver which opened India's account in Paris 1900. KD Jadhav, with a wrestling ..."}, {'url': 'https://economictimes.indiatimes.com

{'input': 'How many medals did India won in olympics 2024?',
 'chat_history': [HumanMessage(content="Hello I'm Vicky"),
  AIMessage(content='Hello Vicky! How can I assist you today?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Vicky. How can I help you, Vicky?'),
  HumanMessage(content='How many medals did India won in olympics?'),
  AIMessage(content='India has won a total of 36 medals in the history of the Olympic Games.')],
 'output': 'India won a total of 1 medal in the 2024 Olympics. Manu Bhaker won the Bronze medal in shooting, becoming the first woman from India to win an Olympic shooting medal.'}